In [155]:
import pandas as pd 
import numpy as np 
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing

In [156]:
# Reading in data
def parse(x):
    return datetime.strptime(x, '%m/%d/%Y')
march2020 = pd.read_csv('march2020clean.csv', sep = ',', date_parser = parse)


# Popping label and date columns
label = march2020.pop('closePrice')

date = march2020.pop('date')

# Dropping features that aren't highly correlated with our label
march2020 = march2020.drop(['openPrice', 'highPrice', 'lowPrice', 'totalSoybeanMealSupply',
         'totalSoybeanMealDemand', 'soybeanOilSupply', 
         'soybeanOilDemand', 'sunflowerSeedPrice', 'peanutsPrice', 'flaxseedPrice',
         'cottonseedOilPrice', 'sunflowerseedOilPrice', 'canolaOilPrice', 'peanutOilPrice',
        'Unnamed: 0'
        ], axis = 1) 

march2020.head()


,Month,canolaPrice,soybeanOilPrice,cornOilPrice,soybeanMealPrice,cottonseedmealPrice,sunflowerseedMealPrice,linseedMealPrice
0,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
1,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
2,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
3,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0
4,November,17.2,33.43,34.46,313.52,228.75,165.0,205.0


In [157]:
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 
          'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10,
         'November': 11, 'December': 12}

Month = list(march2020['Month'])

def encode(l):
    encoded = []
    for m in Month:
        for key, value in months.items():
            if key == m:
                encoded.append(value)
    return encoded

Month = encode(Month)

march2020['Month'] = Month

month = march2020.pop('Month')

march2020.head()

,canolaPrice,soybeanOilPrice,cornOilPrice,soybeanMealPrice,cottonseedmealPrice,sunflowerseedMealPrice,linseedMealPrice
0,17.2,33.43,34.46,313.52,228.75,165.0,205.0
1,17.2,33.43,34.46,313.52,228.75,165.0,205.0
2,17.2,33.43,34.46,313.52,228.75,165.0,205.0
3,17.2,33.43,34.46,313.52,228.75,165.0,205.0
4,17.2,33.43,34.46,313.52,228.75,165.0,205.0


In [158]:
# Normalizing features
x = march2020.values 
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)
march2020 = pd.DataFrame(x_scaled)
march2020['7'] = month 

march2020.head()


,0,1,2,3,4,5,6,7
0,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815,11
1,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815,11
2,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815,11
3,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815,11
4,0.594429,2.744969,2.750389,-0.625971,-0.766518,-0.431963,-1.329815,11


In [159]:
# Train test split
train_index_start = 0
train_index_end = 325 
test_index_start = 326
test_index_end = 407

train_features = march2020.iloc[train_index_start:train_index_end,:]
train_features = train_features.values
train_label = label[train_index_start:train_index_end]
train_label = train_label.values
test_features = march2020.iloc[test_index_start:test_index_end,:]
test_features = test_features.values
test_label = label[test_index_start:test_index_end]
test_label = test_label.values

array([ 978.5 ,  985.  ,  983.  ,  996.5 ,  998.5 ,  998.  , 1003.  ,
       1000.5 , 1001.75, 1001.75, 1000.25,  997.75, 1004.5 , 1004.75,
       1010.  , 1006.25,  997.25,  996.  ,  992.  ,  987.5 ,  990.5 ,
        984.  ,  984.25,  980.25,  976.5 ,  976.  ,  976.25,  976.75,
        984.75,  984.25,  979.5 ,  981.25,  986.25,  987.  ,  986.25,
        988.  ,  986.75,  985.75,  978.75,  977.  ,  989.  ,  991.75,
        991.75,  995.75,  996.  ,  999.  , 1001.25, 1003.75, 1004.75,
       1001.5 , 1004.  , 1007.  , 1004.25,  999.75,  994.  ,  989.  ,
       1000.75,  996.75, 1000.25,  996.  , 1002.75, 1002.75,  999.25,
       1003.75, 1003.  , 1005.25, 1005.75, 1004.75, 1007.25, 1007.75,
       1006.25, 1005.5 , 1005.75, 1000.75, 1004.75, 1012.25, 1012.5 ,
       1012.5 , 1001.75, 1009.  , 1014.25, 1003.  , 1004.75, 1010.  ,
       1002.25, 1004.  , 1003.  , 1002.75, 1001.75, 1001.25,  996.75,
        993.25, 1009.5 , 1000.75, 1000.75,  988.5 ,  996.  ,  995.5 ,
        998.25, 1003